In [38]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import diceroll  # Import the custom module

# Create widgets
dice_face_slider = widgets.SelectionSlider(
    options=[4, 6, 8, 10, 12, 20, 100],
    value=6,  # Default value
    description='Dice Faces:',
    style={'description_width': 'initial'},
    continuous_update=False
)

addon_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Addon:',
    style={'description_width': 'initial'},
    continuous_update=False
)

dice_quantity_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Dice Quantity:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# Create update button
update_button = widgets.Button(
    description='Update Results',
    style={'button_color': 'lightblue'}
)

# Create an Output widget to handle the plot
output = widgets.Output()

# Create a Textarea widget for results display
results_textarea = widgets.Textarea(
    value='',
    placeholder='Results will be displayed here...',
    description='Results:',
    layout=widgets.Layout(width='100%', height='200px'),
    disabled=True
)

# Initialize y_slider (initial values will be set after plotting)
y_slider = widgets.IntSlider(
    value=1,
    min=1,  # Placeholder values
    max=100,
    step=1,
    description='Threshold y:',
    style={'description_width': 'initial'},
    continuous_update=False
)

def update_results(button=None):
    with output:
        clear_output(wait=True)
        dice_faces = dice_face_slider.value
        addon = addon_slider.value
        dice_quantity = dice_quantity_slider.value
        
        dice_prompt = f"{dice_quantity}d{dice_faces}+{addon}"
        outcome_to_probability = diceroll.calculate_pdf(dice_faces, dice_quantity, addon)
        mean_value = diceroll.calculate_mean(outcome_to_probability)
        
        min_value = min(outcome_to_probability.keys())
        max_value = max(outcome_to_probability.keys())
        cumulative_probabilities = []
        
        for outcome, probability in outcome_to_probability.items():
            cumulative_probability = sum([p for x, p in outcome_to_probability.items() if x <= outcome])
            cumulative_probability_percentage = cumulative_probability * 100
            cumulative_probabilities.append(f"P(0<x<{outcome}) = {cumulative_probability_percentage:.2f}%; P({outcome}<x<{max_value}) = {100 - cumulative_probability_percentage:.2f}%")
        
        results_textarea.value = "\n".join(cumulative_probabilities)
        
        # Plot the updated results
        diceroll.plot_values(dice_prompt, outcome_to_probability, mean_value)
        
        # Safely update y_slider based on the current outcome_to_probability
        try:
            # Ensure min and max values are valid and do not overlap
            if min_value < max_value:
                y_slider.min = min_value
                y_slider.max = max_value
                
                # Set default value to within bounds
                y_slider.value = min_value
                
                # Display the slider
                display(y_slider)
            else:
                results_textarea.value += "\nError: min_value is not less than max_value. min_value and max_value are the same or invalid."
        except Exception as e:
            results_textarea.value += f"\nError: {str(e)}"


# Function to handle y_slider changes
def on_y_slider_change(change):
    threshold_y = change['new']
    dice_faces = dice_face_slider.value
    addon = addon_slider.value
    dice_quantity = dice_quantity_slider.value
    
    dice_prompt = f"{dice_quantity}d{dice_faces}+{addon}"
    outcome_to_probability = diceroll.calculate_pdf(dice_faces, dice_quantity, addon)
    min_value = min(outcome_to_probability.keys())
    max_value = max(outcome_to_probability.keys())
    
    # Ensure threshold_y is within bounds
    if min_value <= threshold_y <= max_value:
        cumulative_prob_min_to_y = sum([p for x, p in outcome_to_probability.items() if min_value < x < threshold_y])
        cumulative_prob_y_to_max = sum([p for x, p in outcome_to_probability.items() if threshold_y < x < max_value])
        
        cumulative_prob_min_to_y_percentage = cumulative_prob_min_to_y * 100
        cumulative_prob_y_to_max_percentage = cumulative_prob_y_to_max * 100
        
        results_textarea.value = f"\nFor y = {threshold_y}:\n" \
                                 f"P({min_value}<x<{threshold_y}) = {cumulative_prob_min_to_y_percentage:.2f}%\n" \
                                 f"P({threshold_y}<x<{max_value}) = {cumulative_prob_y_to_max_percentage:.2f}%"
    else:
        results_textarea.value += "\nError: Threshold value is out of bounds."

# Attach functions to events
update_button.on_click(update_results)
y_slider.observe(on_y_slider_change, names='value')

# Display widgets
display(dice_face_slider)
display(addon_slider)
display(dice_quantity_slider)
display(update_button)
display(output)
display(results_textarea)

# Initial call to display results
update_results()

SelectionSlider(continuous_update=False, description='Dice Faces:', index=1, options=(4, 6, 8, 10, 12, 20, 100…

IntSlider(value=1, continuous_update=False, description='Addon:', max=20, min=1, style=SliderStyle(description…

IntSlider(value=1, continuous_update=False, description='Dice Quantity:', max=10, min=1, style=SliderStyle(des…

Button(description='Update Results', style=ButtonStyle(button_color='lightblue'))

Output()

Textarea(value='', description='Results:', disabled=True, layout=Layout(height='200px', width='100%'), placeho…